## Correlations Between Atmospheric CO2 and Land Use Leveraged by Machine Learning
## Jake Kastenbauer

Here a variation of an image detection convolutional neural network (CNN) is presented that determines land cover type by ten distinct classes. This CNN is then used to predict the dominant land cover type using external images that will be used for subsequent analysis. With only five distinct images collected, a variation of image segmentaion is performed on the backend of this notebook, whereby each image is broken up into pixels that are then individually analyzed with the trained image classification model.

# 1. Image Classification Model Development

First, modules are imported for the whole notebook and a land cover image classification dataset, EuroSAT, is loaded.

In [ ]:
# Import modules
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np
import os
from PIL import Image

In [ ]:
# Download land use image detection dataset
ds_train, ds_info = tfds.load('eurosat/rgb', split='train', shuffle_files=True, with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/27000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/eurosat/rgb/2.0.0.incomplete9C1JCV/eurosat-train.tfrecord*...:   0%|      …

Dataset eurosat downloaded and prepared to /root/tensorflow_datasets/eurosat/rgb/2.0.0. Subsequent calls will reuse this data.


Then model training parameters are established, describing the extent of the dataset used for training and validation, along with data augmentation and processessing.

In [ ]:
# Develop CNN pipeline
# Note batchsize, percent of data for training, and augmentations
BATCH_SIZE = 16

AUTO = tf.data.experimental.AUTOTUNE
SHUFFLE_BUFFER = int(ds_info.splits['train'].num_examples * 0.8)

# Split our data into training (80%) and validation (20%)
ds_train, ds_valid = tfds.load('eurosat/rgb',
                               split=['train[:80%]', 'train[80%:]'],
                               as_supervised=True)

# CNN Training Mechanics
# Apply data augmentation and preprocessing
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    return image, label

def preprocess(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label


ds_train = (ds_train
            .map(preprocess, AUTO)
            .cache()
            .shuffle(SHUFFLE_BUFFER)
            .repeat()
            .map(augment, AUTO)
            .batch(BATCH_SIZE, drop_remainder=True)
            .prefetch(AUTO))

ds_valid = (ds_valid
            .map(preprocess, AUTO)
            .cache()
            .batch(BATCH_SIZE)
            .prefetch(AUTO))

Model architecture is then constructed with multiple layers of 2D convolution and max pooling, followed by compiling.

In [ ]:
# Create model
# Label classes
NUM_CLASSES = ds_info.features['label'].num_classes

# Build the model
# Implement 2D convolution and max poolinglayers
model = keras.Sequential([
    layers.BatchNormalization(),
    layers.Conv2D(filters=16, kernel_size=5, padding='same', activation='elu'),
    layers.MaxPool2D(),

    layers.BatchNormalization(),
    layers.Conv2D(32, 3, padding='same', activation='elu'),
    layers.Conv2D(32, 3, padding='same', activation='elu'),
    layers.MaxPool2D(),

    layers.BatchNormalization(),
    layers.Conv2D(64, 3, padding='same', activation='elu'),
    layers.Conv2D(64, 3, padding='same', activation='elu'),
    layers.MaxPool2D(),

    layers.BatchNormalization(),
    layers.Conv2D(128, 3, padding='same', activation='elu'),
    layers.MaxPool2D(),

    layers.Flatten(),
    layers.Dense(128, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(64, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax')])

# Compile
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],)

# 2. Model Training

Model training is executed based on training and validation parameters, and architecture steps outlined above.

In [ ]:
# Set parameters to execute model training
EPOCHS = 5
STEPS_PER_EPOCH = int(ds_info.splits['train'].num_examples * 0.8) // BATCH_SIZE

print("Epochs:", EPOCHS)
print("Steps per Epoch:", STEPS_PER_EPOCH)


early_stopping = tf.keras.callbacks.EarlyStopping(patience=7, min_delta=0.001, restore_best_weights=True)

# Train model
history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=[early_stopping],)

Epochs: 5
Steps per Epoch: 1350
Epoch 1/5
1350/1350 [==============================] - 297s 215ms/step - loss: 1.8297 - sparse_categorical_accuracy: 0.4026 - val_loss: 1.2149 - val_sparse_categorical_accuracy: 0.5613
Epoch 2/5
1350/1350 [==============================] - 280s 208ms/step - loss: 1.1918 - sparse_categorical_accuracy: 0.5827 - val_loss: 0.7222 - val_sparse_categorical_accuracy: 0.7154
Epoch 3/5
1350/1350 [==============================] - 282s 209ms/step - loss: 0.9825 - sparse_categorical_accuracy: 0.6627 - val_loss: 0.6072 - val_sparse_categorical_accuracy: 0.7852
Epoch 4/5
1350/1350 [==============================] - 285s 211ms/step - loss: 0.8353 - sparse_categorical_accuracy: 0.7272 - val_loss: 0.5591 - val_sparse_categorical_accuracy: 0.8224
Epoch 5/5
1350/1350 [==============================] - 288s 213ms/step - loss: 0.7406 - sparse_categorical_accuracy: 0.7621 - val_loss: 0.6314 - val_sparse_categorical_accuracy: 0.7952


In [ ]:
# Save model
model.save('model.keras')

# 3. Model Implementation for External Data Use

The model is used to assess the land cover class of an image on several levels. First as the image as a whole.

In [55]:
# Call back model to post apply a new image for classification
# Load and preprocess the image
img_path = 'Photo1.jpg'
img = image.load_img(img_path, target_size=(64, 64))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# Get the model's prediction
predictions = model.predict(img_array)

# Map class indices to class labels
class_labels = {0: 'AnnualCrop', 1: 'SeaLake', 2: 'Industiral', 3: 'Highway', 4: 'HerbaceousVegetation', 5: 'Residential', 6: 'PermanentCrop', 7: 'Pasture', 8: 'River', 9: 'Forest'}

# Find the predicted class index
predicted_class_index = np.argmax(predictions[0])

# Get the predicted class label
predicted_class_label = class_labels[predicted_class_index]

# Print the predicted class label
print("Predicted class:", predicted_class_label)

1/1 [==============================] - 0s 202ms/step
Predicted class: Forest


This can include likelihood estimates of an image belonging to each class.

In [56]:
# Extract predictions for the image
single_prediction = predictions[0]

# Get the top-3 predicted classes and their probabilities
top_classes = np.argsort(single_prediction)[-3:][::-1]
top_probabilities = single_prediction[top_classes]

# Decode and print the top-3 predicted classes
for i, (class_index, probability) in enumerate(zip(top_classes, top_probabilities)):
    print(f"{i + 1}: Class {class_index} with Probability {probability * 100:.2f}%")

1: Class 9 with Probability 99.99%
2: Class 8 with Probability 0.01%
3: Class 5 with Probability 0.00%


With only five images, perhaps the best option for more robust correlative analysis is to assess each pixel of the image individually.

In [57]:
# Resize an image to the specifications of the model
def resize_image(input_path, output_path, size):
    image = Image.open(input_path)
    resized_image = image.resize(size)
    resized_image.save(output_path)

# Segment into pixels in an array
def segment_image(image_path, output_folder, model, num_classes, perc=0.1):
    image = Image.open(image_path)
    image_array = np.array(image)
    height, width, _ = image_array.shape
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    class_counts = np.zeros(num_classes)

    # Save each pixel as a separate image in the output folder with a given class
    for i in range(height):
        for j in range(width):
            pixel_value = image_array[i, j]
            if np.random.rand() < perc:
                predicted_class = np.random.randint(num_classes)
            else:
                pixel_image_array = np.ones((64, 64, 3), dtype=np.uint8) * pixel_value
                pixel_image_reshaped = pixel_image_array.reshape(1, 64, 64, 3)
                prediction = model.predict(pixel_image_reshaped)
                predicted_class = np.argmax(prediction)

            class_counts[predicted_class] += 1
            pixel_image_path = os.path.join(output_folder, f"pixel_{i}_{j}_class_{predicted_class}.png")
            Image.fromarray(pixel_image_array).save(pixel_image_path)

    # Calculate the percentage of pixels for each class
    total_pixels = height * width
    class_percentages = (class_counts / total_pixels) * 100
    print("Class Percentages:", class_percentages)

Here, pixel by pixel image classification can be done for a single imported image. The image paths and folders can be changed to facilitate multiple image analyses.

In [58]:
# Example with a single external image
input_image_path = "Photo1.jpg"
resized_image_path = "Photo1_1.jpg"
output_folder_path = "Photo1_Pixels"

# Load model and parameters
model = keras.models.load_model("model.keras")
num_classes =  ds_info.features['label'].num_classes
perc = 0.15

# Resize and segment image
resize_image(input_image_path, resized_image_path, (64, 64))
segment_image(resized_image_path, output_folder_path, model, num_classes, perc)

1/1 [==============================] - 0s 68ms/step
Class Percentages: [ 2.51464844  2.51464844  2.1484375   2.63671875  2.49023438  2.09960938
  2.49023438  2.46582031  1.953125   78.68652344]


This project seeks to utilize a trained CNN to determine land cover types for basic imported land cover images, and give an approximate assessment of the likelihood for a given class for individual image pixels. Satellites such as OCO-2 have been used to determine large scale changes in carbon based GHGs, but little information is present on how smaller scale changes in land cover have impacted detectable GHG levels when alanlyzed using satellite remote sensing technology. Here we see steps toward land cover classification which may be of use in correlative analyses with CO2. Along with greenhouse gas, data this can be used to obtain a robust assessment of how major land cover classes influence atmospheric CO2 dynamics on a smaller regional scale.